# AML — Task 1
## Predict the age of a brain from MRI features
---

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from pandas_profiling import ProfileReport

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression, mutual_info_regression, SelectKBest
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.neighbors import LocalOutlierFactor

In [3]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

## Import datasets

In [4]:
def import_data(folder="data/", extension=""):
    X_train = pd.read_csv(folder + 'X_train' + extension + '.csv').drop(columns=['id'])
    y_train = pd.read_csv(folder + 'y_train' + extension + '.csv').drop(columns=['id'])
    X_test = pd.read_csv(folder + 'X_test' + extension + '.csv').drop(columns=['id'])
    return X_train, y_train, X_test

---
## Exporting Data as csv

In [5]:
def export_to_csv(X_train_cleaned, y_train_cleaned, X_test_cleaned, folder="data/"):
    X_train_cleaned.to_csv(folder + 'X_train_cleaned.csv', index=False)
    y_train_cleaned.to_csv(folder + 'y_train_cleaned.csv', index=False)
    X_test_cleaned.to_csv(folder + 'X_test_cleaned.csv', index=False)

---
## Task 1: Outlier detection

In [6]:
"""Remove the ouliers from our dataset. Replace temporarily the Nan values by the mean to perform outlier selection

Parameters
----------
X_train : pd.df
    The features (what we will use to see the outliers)
y_train : pd.df
    The labels
contamination : int, optional
    The percent of outliers found by the isolation forest if it's used.
verbose : int, optional
    If not set to 0, print messages
    
Return
------
(pd.df, pd.df)
    The data with the outliers rows removed
"""
def remove_outliers(X_train, y_train, contamination='auto', verbose=1, method="LocalOutlierFactor"):
    # Save a mask of the imputed values to be able to redo the imputation once the outlier detection is done
    X_train_null_mask = X_train.isna()
    
    # Need to impute nan values for the outlier detection to work (cannot deal with nan)
    X_train_imputed = pd.DataFrame(SimpleImputer(strategy="median", verbose=verbose).fit_transform(X_train))
    
    clf = None
    if method=="LocalOutlierFactor":
        clf = LocalOutlierFactor(contamination=contamination)
    elif method=="IsolationForest":
        clf = IsolationForest(contamination=contamination, random_state=0, verbose=verbose)
    else:
        raise AttributeError(f"Unvalid argument for method, must be 'LocalOutlierFactor' or 'IsolationForest', not '{method}'")
        
    outliers_mask = pd.Series(clf.fit_predict(X_train_imputed)).map({1:1, -1:0}) #Mask with 0 for outliers and 1 for non outliers
    
    if verbose:
        print(f"Detected {(outliers_mask == 0).sum()} outliers with method {method}, out of {outliers_mask.shape[0]} samples ({100 * (outliers_mask == 0).sum() / outliers_mask.shape[0]:.2f}%).")
    
    #Replace the Nan values (The outlier detection shouldn't replace NaN values by itself)
    X_train = pd.DataFrame(X_train).mask(X_train_null_mask, other=np.NaN, inplace=False)
    
    # Remove outliers from the training set
    X_train = np.array(X_train)[outliers_mask == 1, :]
    y_train = np.array(y_train)[outliers_mask == 1, :]
    
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    
    return (X_train, y_train)

---
## Task 2: Data scaling
Done as soon as possible because can have an effect (e.g. on distances for `KNNImputer`)

In [7]:
def scale(X_train_no_outliers, X_test):
    # Do the scaling, saving the scaler to use it for X_test too. No need imputation, just ignore Nan values
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train_no_outliers))
    # Cast X_test to np.array to avoid warning of model trained without feature names but X having some.
    X_test_scaled = pd.DataFrame(scaler.transform(np.array(X_test)))
    return (X_train_scaled, X_test_scaled)

---
## Task 3: impute values

In [8]:
def impute_values(X_train, X_test, method='knn', max_iter=15):
    print(f"For the train dataset, there are {np.array(X_train.isna()).sum().sum()} nan values, out of {X_train.shape[0]*X_train.shape[1]} ({100*np.array(X_train.isna()).sum().sum()/(X_train.shape[0]*X_train.shape[1]):.2f}%).")
    
    imputer = None
    if method == 'knn':
        imputer = KNNImputer(n_neighbors=6, weights='uniform').fit(X_train)
    elif method == 'iterative':
        # Runs VERY slowly and might cause the kernel to crash due to intenisve use of RAM
        imputer = IterativeImputer(random_state=0, max_iter=max_iter, verbose=2, initial_strategy='median').fit(X_train)
    else:
        raise AttributeError(f"Unvalid argument for method, must be 'knn' or 'iterative', not '{method}'")
    
    X_train_imputed = pd.DataFrame(imputer.transform(X_train))
    X_test_imputed = pd.DataFrame(imputer.transform(X_test))
    return (X_train_imputed, X_test_imputed)

---
## Task 4: Feature selection

In [9]:
def select_features(X_train_imputed, X_test_imputed):
    X_train_selected_features, X_test_selected_features = remove_constant_features(X_train_imputed, X_test_imputed)
    X_train_selected_features, X_test_selected_features = remove_too_coorelated_features(X_train_selected_features, X_test_selected_features)
    X_train_selected_features, X_test_selected_features = remove_random_features(X_train_selected_features, y_train, X_test_selected_features, percentile=80)

    return X_train_selected_features, X_test_selected_features

In [10]:
def remove_constant_features(X_train_imputed, X_test_imputed, verbose=1):
    X_train_selected_features = X_train_imputed.loc[:, np.logical_and(X_train_imputed != X_train_imputed.iloc[0], X_train_imputed.notna()).any()]
    X_test_selected_features = X_test_imputed.loc[:, np.logical_and(X_train_imputed != X_train_imputed.iloc[0], X_train_imputed.notna()).any()]
    
    if verbose:
        print(f"{X_train_imputed.shape[1]-X_train_selected_features.shape[1]} features removed because of constant values ({100*(X_train_imputed.shape[1]-X_train_selected_features.shape[1])/X_train_imputed.shape[1]:.2f}%)")
    
    return X_train_selected_features, X_test_selected_features

In [11]:
def remove_too_coorelated_features(X_train, X_test, threshold=0.7, verbose=1):
    X_train_corr_ = X_train.corr()

    X_train_too_correlated = (X_train_corr_.mask(
        np.tril(np.ones([len(X_train_corr_)]*2, dtype=bool))).abs() > threshold).any()
    
    
    X_train_selected_features = X_train.loc[:, (~X_train_too_correlated)]
    X_test_selected_features = X_test.loc[:, (~X_train_too_correlated)]
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of correlation > {threshold} ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%)")

    return X_train_selected_features, X_test_selected_features

In [12]:
def remove_random_features(X_train, y_train, X_test, Xtrm=None, Xtem=None, k=190, verbose=1):
    selector = SelectKBest(f_regression, k=k) # modify here
    selector.fit(X_train, np.array(y_train).ravel())
    X_train_selected_features = pd.DataFrame(selector.transform(X_train))
    X_test_selected_features = pd.DataFrame(selector.transform(X_test))
    if Xtrm is not None:
        Xtrm = pd.DataFrame(selector.transform(Xtrm))
    if Xtem is not None:
        Xtem = pd.DataFrame(selector.transform(Xtem))
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of low correlation with target ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%).")
        
    return X_train_selected_features, X_test_selected_features, Xtrm, Xtem

---
## Task 5 Models

In [13]:
def model(X_train_cleaned, y_train_cleaned):
    gs_lasso = best_lasso(X_train_cleaned, y_train_cleaned)
    gs_svr = best_svr(X_train_cleaned, y_train_cleaned)
    gs_gbr = best_gbr(X_train_cleaned, y_train_cleaned)
    gs_ada = best_Adaboost(X_train, y_train)
    gs_rforest = best_random_forest(X_train, y_train)
    max_score = max(max(max(max(gs_gbr.best_score_, gs_lasso.best_score_), gs_svr.best_score_), gs_ada.best_score_), gs_rforest.best_score)

    return gs_lasso if gs_lasso.best_score_ == max_score else gs_svr if gs_svr.best_score_ == max_score else gs_ada if gs_ada.best_score_ == max_score else gs_rforest if gs_rforest.best_score == max_score else gs_gbr

### Model 1: Lasso

In [14]:
def best_lasso(X_train_cleaned, y_train_cleaned):
    lasso = Lasso(max_iter=100000)
    gs_lasso_params = {
    'alpha': np.logspace(-1, 0, 20),
    }
    gs_lasso = GridSearchCV(lasso, gs_lasso_params, cv=5, verbose=3)
    gs_lasso.fit(X_train_cleaned, y_train_cleaned)
    print(f"The best validation score obtained is {gs_lasso.best_score_:.5f} with\n\talpha: {gs_lasso.best_params_['alpha']}")
    return gs_lasso;

### Model 2: SVR (SVM for regression)

In [15]:
def best_svr(X_train_cleaned, y_train_cleaned):
    svr = SVR()
    gs_svr_params = {
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'C': np.logspace(-1, 2.2, 4),
    'epsilon': np.logspace(-2, 1, 3),
    }
    gs_svr = GridSearchCV(svr, gs_svr_params, cv=5, verbose=3)
    gs_svr.fit(X_train_cleaned, y_train_cleaned)
    print(f"""The best validation score obtained is {gs_svr.best_score_:.5f} with
    \tkernel: {gs_svr.best_params_['kernel']}
    \tC: {gs_svr.best_params_['C']}
    \tepsilon: {gs_svr.best_params_['epsilon']}""")
    return gs_svr

### Model 3: Gradient Boosting

In [16]:
def best_gbr(X_train, y_train):
    gbr = GradientBoostingRegressor()
    gs_gbr_params = {
     "loss":["ls"],
     "learning_rate": np.logspace(-2, -1, 3),
     "n_estimators": np.arange(50, 400, 50),
     "subsample": np.arange(0.4, 1, 0.2),
     "max_depth": np.arange(2, 8, 2),
     "min_samples_split": np.arange(3, 9, 2),
     "min_samples_leaf": np.arange(1, 9, 2),
     "random_state": [0], 
     "verbose": [1]
    }
    gs_gbr = GridSearchCV(gbr, gs_gbr_params, cv=4, verbose=2, error_score='raise')
    gs_gbr.fit(X_train, y_train)

    print(f"""The best validation score obtained is {gs_gbr.best_score_:.5f} with
    \tloss: {gs_gbr.best_params_['loss']}
    \tlearning_rate: {gs_gbr.best_params_['learning_rate']}
    \tn_estimators: {gs_gbr.best_params_['n_estimators']}
    \tsubsample: {gs_gbr.best_params_['subsample']}
    \tmax_depth: {gs_gbr.best_params_['max_depth']}
    \tmin_samples_split: {gs_gbr.best_params_['min_samples_split']}
    \tmin_samples_leaf: {gs_gbr.best_params_['min_samples_leaf']}
    """)
        
    return gs_gbr

In [17]:
def old_best_gbr(X_train, y_train):

    params = {
        "loss":"ls",
        "n_estimators": 250,
        "learning_rate": 0.025,
        "subsample": 0.75,
        "max_depth": 6,
        "min_samples_split": 5,
        "n_iter_no_change": 100,
        "validation_fraction": 0.1,
        "random_state": 0, 
        "verbose": 1,
    }

    gbr = GradientBoostingRegressor(**params)
    
    # Get the validation score
    gbr_cv_scores = cross_val_score(gbr, X_train, y_train, n_jobs=-1, verbose=3)
    print(f"""Validation score obtained is {np.mean(gbr_cv_scores):.4f} with
    \t{params}""")
    
    # Fit model (because previous function does not return fitted model)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    gbr.fit(X_train, y_train)
    
    test_score = np.zeros(gbr.n_estimators_, dtype=np.float64)
    for i, y_pred in enumerate(gbr.staged_predict(X_test)):
        test_score[i] = gbr.loss_(y_test, y_pred)

    fig = plt.figure(figsize=(6, 6))
    plt.subplot(1, 1, 1)
    plt.title("Deviance")
    plt.plot(
        np.arange(gbr.n_estimators_),
        gbr.train_score_,
        "b-",
        label="Training Set Deviance",
    )
    plt.plot(
        np.arange(gbr.n_estimators_), test_score, "r-", label="Test Set Deviance"
    )
    plt.legend(loc="upper right")
    plt.xlabel("Boosting Iterations")
    plt.ylabel("Deviance")
    fig.tight_layout()
    plt.show()
    
    return gbr

## Model 4: Adaboost

In [18]:
def best_Adaboost(X_train, y_train):
    
    ada = AdaBoostRegressor()
    gs_ada_params = {
     "loss":["square"],
     "learning_rate": np.logspace(-2, -1, 5),
     "n_estimators": np.arange(50, 550, 100),
     "random_state": [0], 
    }
    gs_ada = GridSearchCV(ada, gs_ada_params, cv=5, verbose=3, error_score='raise')
    gs_ada.fit(X_train, y_train)

    print(f"""The best validation score obtained is {gs_ada.best_score_:.5f} with
    \tloss: {gs_ada.best_params_['loss']}
    \tlearning_rate: {gs_ada.best_params_['learning_rate']}
    \tn_estimators: {gs_ada.best_params_['n_estimators']}
    """)
    
    return ada

## Model 5: Random Forest

In [19]:
def best_random_forest(X_train, y_train):
    # define used regressor
    forest=RandomForestRegressor(n_estimators=20,
                                 max_depth=10,
                                 criterion='mse',)

    r_forest = RandomForestRegressor()
    gs_forest_params = {
     "n_estimators": np.arange(50, 500, 50),
     "max_depth": np.arange(2, 8, 1),
     "min_samples_split": np.arange(2, 8, 1),
     "min_samples_leaf": np.arange(1, 9, 2),
     "random_state": [0], 
     "verbose": [1]
    }
    
    gs_forest = GridSearchCV(r_forest, gs_forest_params, cv=5, verbose=3, error_score='raise')
    gs_forest.fit(X_train, y_train)
    

    print(f"""The best validation score obtained is {gs_forest.best_score_:.5f} with
    \tn_estimators: {gs_forest.best_params_['n_estimators']}
    \tmax_depth: {gs_forest.best_params_['max_depth']}
    \tmin_samples_split: {gs_forest.best_params_['min_samples_split']}
    \tmin_samples_leaf: {gs_forest.best_params_['min_samples_leaf']}
    """)
    
    return r_forest

---
## Create CSV for submission

In [20]:
def create_submission(prediction, sub_id, basepath = 'submissions/task1-sub'):
    result = prediction.copy()
    result = result.rename(columns={0: 'y'})
    result['id'] = range(0, len(result))
    result = result[['id', 'y']]
    result.to_csv(basepath+str(sub_id) + '.csv', index=False)

---
## Final Pipeline

In [21]:
print("Loading raw data...")
X_train, y_train, X_test = import_data()

print("Removing outliers...")
X_train_no_outliers, y_train_cleaned = remove_outliers(X_train, y_train, method="IsolationForest")
X_train_no_outliers, y_train_cleaned = remove_outliers(X_train_no_outliers, y_train_cleaned, method="LocalOutlierFactor") #Remove outliers in x


print("Scaling data...")
X_train_scaled, X_test_scaled = scale(X_train_no_outliers, X_test)

print("Selecting features...")
X_train_no_constant_features, X_test_no_constant_features = remove_constant_features(X_train_scaled, X_test_scaled)
X_train_no_too_correlated_features, X_test_no_too_correlated_features = remove_too_coorelated_features(X_train_no_constant_features, X_test_no_constant_features, threshold=0.98)

#TODO: Clean pipeline
# If we want iterative imputation, and don't want to run it for one hour (should try at one point though), need to
# do feature selection before we do imputation. Except biggest filter for feature selection is f_regression which
# cannot deal with nan values. Therefore, intermediary dumb knn imputation was quickly/dirtily 
# implemented to see results. However, I tried to run Iterative Imputations for just a few iterations and it didn't give
# better results. Furthermore, we need more than 16GB RAM if we want to run it. Solutions could be to use google colab
# to access GPU.
# Conclusion: we achieve ~same or better validation scores with a LOT less features(random features), seems like an 
# interresting way forward. However, we shouldn't remove too many features with high correlations, cause it makes
# the score being worse.

#We first impute with knn which is cheap and then later on reaply mask and use iterative imputing
X_train_mask = X_train_no_too_correlated_features.isna()
X_test_mask = X_test_no_too_correlated_features.isna()
X_train_knn_imputed, X_test_knn_imputed = impute_values(X_train_no_too_correlated_features, X_test_no_too_correlated_features, method='knn')
X_train_selected_features, X_test_selected_features, X_train_mask, X_test_mask = remove_random_features(X_train_knn_imputed, y_train_cleaned, X_test_knn_imputed, X_train_mask, X_test_mask, k=190)
X_train_no_imputation = X_train_selected_features.mask(np.array(X_train_mask))
X_test_no_imputation = X_test_selected_features.mask(np.array(X_test_mask))

print("Imputing nan values...")
X_train_cleaned, X_test_cleaned = impute_values(X_train_no_imputation, X_test_no_imputation, method='iterative')

print("Exporting clean data to csv...")
export_to_csv(X_train_cleaned, y_train_cleaned, X_test_cleaned)
#export_to_csv(X_train_knn_imputed, y_train_cleaned, X_test_knn_imputed)

print("All done!")

Loading raw data...
Removing outliers...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


Detected 5 outliers with method IsolationForest, out of 1212 samples (0.41%).
Detected 49 outliers with method LocalOutlierFactor, out of 1207 samples (4.06%).
Scaling data...
Selecting features...
3 features removed because of constant values (0.36%)
35 features removed because of correlation > 0.98 (4.22%)
For the train dataset, there are 70105 nan values, out of 919452 (7.62%).


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


604 features removed because of low correlation with target (76.07%).
Imputing nan values...
For the train dataset, there are 16741 nan values, out of 220020 (7.61%).
[IterativeImputer] Completing matrix with shape (1158, 190)
[IterativeImputer] Ending imputation round 1/15, elapsed time 2.21
[IterativeImputer] Change: 46.64924800588054, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 2/15, elapsed time 4.26
[IterativeImputer] Change: 4.921670143727958, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 3/15, elapsed time 6.30
[IterativeImputer] Change: 5.0086204610579514, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 4/15, elapsed time 8.43
[IterativeImputer] Change: 1.3793821714844152, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 5/15, elapsed time 10.65
[IterativeImputer] Change: 0.5430808321288869, scaled tolerance: 0.014114220441223293 
[Ite

/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


[IterativeImputer] Ending imputation round 4/15, elapsed time 0.24
[IterativeImputer] Ending imputation round 5/15, elapsed time 0.30
[IterativeImputer] Ending imputation round 6/15, elapsed time 0.35
[IterativeImputer] Ending imputation round 7/15, elapsed time 0.41
[IterativeImputer] Ending imputation round 8/15, elapsed time 0.46
[IterativeImputer] Ending imputation round 9/15, elapsed time 0.52
[IterativeImputer] Ending imputation round 10/15, elapsed time 0.58
[IterativeImputer] Ending imputation round 11/15, elapsed time 0.63
[IterativeImputer] Ending imputation round 12/15, elapsed time 0.69
[IterativeImputer] Ending imputation round 13/15, elapsed time 0.74
[IterativeImputer] Ending imputation round 14/15, elapsed time 0.80
[IterativeImputer] Ending imputation round 15/15, elapsed time 0.85
[IterativeImputer] Completing matrix with shape (776, 190)
[IterativeImputer] Ending imputation round 1/15, elapsed time 0.04
[IterativeImputer] Ending imputation round 2/15, elapsed time 0.

In [22]:
#gs = model(X_train_cleaned, np.array(y_train_cleaned).ravel())

In [ ]:
gs = best_gbr(X_train_cleaned, np.array(y_train_cleaned).ravel())

Fitting 4 folds for each of 2268 candidates, totalling 9072 fits
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          81.8234           0.6979            0.48s
         2          96.1339           0.6076            0.48s
         3          81.8230           0.5519            0.46s
         4          91.1700           0.5811            0.44s
         5          88.0238           0.6124            0.43s
         6          95.7475           0.6231            0.42s
         7          88.9520           0.6293            0.40s
         8          97.7350           0.6249            0.39s
         9          92.7502           0.5564            0.38s
        10          88.1625           0.5229            0.37s
        20          76.4206           0.5251            0.27s
        30          77.9474           0.3955            0.18s
        40          71.9411           0.3449            0.09s
        50          66.5207           0.3021            0.00s
[CV]

        30          70.9312           0.3312            0.26s
        40          64.2117           0.2417            0.13s
        50          63.8133           0.3020            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=1, min_samples_split=3, n_estimators=50, random_state=0, subsample=0.6000000000000001, verbose=1; total time=   0.7s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          94.2018           0.6472            0.91s
         2          87.7652           0.7968            0.89s
         3          88.4423           0.6839            0.87s
         4          89.7403           0.6799            0.85s
         5          90.5372           0.5745            0.83s
         6          93.1239           0.5006            0.81s
         7          85.5485           0.5946            0.79s
         8          88.9948           0.5395            0.77s
         9          91.2836           0.4278            0.75s
        

        30          70.4805           0.3995            0.60s
        40          68.4332           0.2284            0.51s
        50          61.9519           0.2888            0.42s
        60          59.5630           0.2513            0.33s
        70          56.7928           0.2580            0.25s
        80          57.0020           0.1716            0.17s
        90          55.9957           0.1642            0.08s
       100          52.2358           0.1320            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=1, min_samples_split=3, n_estimators=100, random_state=0, subsample=0.4, verbose=1; total time=   0.8s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1         100.4121           0.6670            0.79s
         2          89.6553           0.7044            0.79s
         3          93.6026           0.6357            0.78s
         4          83.6144           0.5284            0.78s
         5          86

        10          87.4924           0.5412            1.83s
        20          81.1594           0.4649            1.55s
        30          75.7498           0.4663            1.32s
        40          72.2535           0.2725            1.10s
        50          66.9491           0.3485            0.90s
        60          63.3018           0.2731            0.71s
        70          63.3485           0.2119            0.53s
        80          56.3144           0.1602            0.35s
        90          58.0066           0.1978            0.17s
       100          52.9833           0.1474            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=1, min_samples_split=3, n_estimators=100, random_state=0, subsample=0.8000000000000002, verbose=1; total time=   1.7s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          98.1504           0.6649            1.66s
         2          94.3936           0.7565            1.66s
       

        30          70.4805           0.3995            0.97s
        40          68.4332           0.2284            0.88s
        50          61.9519           0.2888            0.80s
        60          59.5630           0.2513            0.72s
        70          56.7928           0.2580            0.64s
        80          57.0020           0.1716            0.56s
        90          55.9957           0.1642            0.48s
       100          52.2358           0.1320            0.40s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=1, min_samples_split=3, n_estimators=150, random_state=0, subsample=0.4, verbose=1; total time=   1.2s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1         100.4121           0.6670            1.16s
         2          89.6553           0.7044            1.16s
         3          93.6026           0.6357            1.16s
         4          83.6144           0.5284            1.15s
         5          86

...

        30          72.0701           0.3495            1.35s
        40          67.1437           0.3408            1.26s
        50          62.1581           0.3125            1.18s
        60          60.2055           0.2830            1.09s
        70          60.4316           0.2383            1.01s
        80          58.4832           0.2318            0.94s
        90          55.5935           0.1823            0.86s
       100          54.4833           0.1771            0.78s
       200          39.2697           0.0593            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=200, random_state=0, subsample=0.4, verbose=1; total time=   1.6s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          91.7695           0.6756            2.39s
         2          89.0459           0.6861            2.38s
         3          85.4921           0.7058            2.38s
         4          89

        20          84.7648           0.4769            3.20s
        30          78.1311           0.4137            2.98s
        40          73.5702           0.2875            2.78s
        50          68.2967           0.4301            2.58s
        60          63.4887           0.2559            2.40s
        70          66.2145           0.1789            2.22s
        80          56.8169           0.2454            2.05s
        90          56.5793           0.1805            1.88s
       100          54.3474           0.1455            1.72s
       200          40.9297           0.0645            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=200, random_state=0, subsample=0.8000000000000002, verbose=1; total time=   3.6s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          90.9021           0.4628            3.59s
         2          87.1856           0.6116            3.55s
       

        30          72.0701           0.3495            1.71s
        40          67.1437           0.3408            1.63s
        50          62.1581           0.3125            1.55s
        60          60.2055           0.2830            1.48s
        70          60.4316           0.2383            1.40s
        80          58.4832           0.2318            1.32s
        90          55.5935           0.1823            1.24s
       100          54.4833           0.1771            1.16s
       200          39.2697           0.0593            0.39s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=250, random_state=0, subsample=0.4, verbose=1; total time=   1.9s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          91.7695           0.6756            2.98s
         2          89.0459           0.6861            2.97s
         3          85.4921           0.7058            2.97s
         4          89

        20          84.7648           0.4769            3.98s
        30          78.1311           0.4137            3.76s
        40          73.5702           0.2875            3.58s
        50          68.2967           0.4301            3.39s
        60          63.4887           0.2559            3.22s
        70          66.2145           0.1789            3.05s
        80          56.8169           0.2454            2.88s
        90          56.5793           0.1805            2.70s
       100          54.3474           0.1455            2.54s
       200          40.9297           0.0645            0.84s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=250, random_state=0, subsample=0.8000000000000002, verbose=1; total time=   4.2s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          90.9021           0.4628            4.17s
         2          87.1856           0.6116            4.16s
       

        30          72.0701           0.3495            2.26s
        40          67.1437           0.3408            2.17s
        50          62.1581           0.3125            2.09s
        60          60.2055           0.2830            2.01s
        70          60.4316           0.2383            1.92s
        80          58.4832           0.2318            1.84s
        90          55.5935           0.1823            1.75s
       100          54.4833           0.1771            1.67s
       200          39.2697           0.0593            0.81s
       300          34.5181          -0.0067            0.00s
[CV] END learning_rate=0.01, loss=ls, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=300, random_state=0, subsample=0.4, verbose=1; total time=   2.4s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          91.7695           0.6756            3.89s
         2          89.0459           0.6861            3.88s
         3          85

In [ ]:
prediction = pd.DataFrame(gs.predict(X_test_cleaned))
SUB_ID = 18 #to modify
create_submission(prediction, SUB_ID)